In [1]:
import os 

import torch
import torch.nn as nn 
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from helpers import load_experiment

In [9]:
# baseline /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-11/18-14-45
# framework /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-11/19-47-58


dm, model = load_experiment("/home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-11/19-47-58")

Global seed set to 42


Fetching configuration...
Loading datamodule...


/home/danis/anaconda3/envs/bottleneck/lib/python3.10/site-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(
100%|██████████| 2700/2700 [00:00<00:00, 9638.05it/s]


Len of vocab:  54
Max len of caption:  16
Index for <pad>: [0]
Loading model


/home/danis/anaconda3/envs/bottleneck/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'criterion_task' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['criterion_task'])`.
  rank_zero_warn(
/home/danis/anaconda3/envs/bottleneck/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'criterion_tie' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['criterion_tie'])`.
  rank_zero_warn(


In [10]:
train_loader = dm.train_dataloader()
train_set = train_loader.dataset

In [11]:
# [red, green, blue, square, triangle, circle]

attribute_mapping = {
    0: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0
        ],
    1: [0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0
        ],
    2: [0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0
        ],
    3: [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0
        ],
    4: [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0
        ],
    5: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0
        ],
    6: [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0
        ],
    7: [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0
        ],
    8: [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1
        ],
}

In [13]:
sample = train_set[50]
image = sample["image"]

image = image.unsqueeze(0).cuda()


model.main.inference(image)[1].cpu().detach().numpy()[0]
# model(image)["concept_probs"].cpu().detach().numpy()[0]

array([9.9804342e-01, 4.7186921e-05, 9.9712926e-01, 3.9565063e-04,
       1.3690232e-05, 9.9684083e-01, 9.9863881e-01, 9.6707660e-01,
       9.6945536e-01, 9.4626157e-05], dtype=float32)

In [13]:
model(image)

TypeError: LitAutoConceptBottleneckModel.forward() missing 1 required positional argument: 'indices'

In [12]:
model(image, torch.tensor([[0, 10, 1]]).cuda())

{'feature_logits': tensor([[  4.3458,   5.8428,   8.8582,   8.2145, -14.4703,  -7.2760,   6.2475,
           -8.1794,   5.8259,   6.3342]], device='cuda:0',
        grad_fn=<AddmmBackward0>),
 'feature_probs': tensor([[9.8720e-01, 9.9711e-01, 9.9986e-01, 9.9973e-01, 5.1956e-07, 6.9146e-04,
          9.9807e-01, 2.8029e-04, 9.9706e-01, 9.9823e-01]], device='cuda:0',
        grad_fn=<SigmoidBackward0>),
 'feature_activated': tensor([[1., 1., 1., 1., 0., 0., 1., 0., 1., 1.]], device='cuda:0',
        grad_fn=<AddBackward0>),
 'prediction': tensor([[ 3.0633, -0.6596, -0.2949,  0.0044, -0.3722, -0.4236, -0.4282, -0.2316,
          -0.3738]], device='cuda:0', grad_fn=<NativeBatchNormBackward0>)}

In [14]:
scores = list()

for feature_id in range(10):

    attribute_values = [[0, 0] for _ in range(len(attribute_mapping[0]))]

    for i in tqdm(range(len(train_set))):
        sample = train_set[i]
        
        idx = sample["target"].item()
        attributes = np.array(attribute_mapping[idx])

        image = sample["image"]
        image = image.unsqueeze(0).cuda()

        feature = model.main.inference(image)[1].cpu().detach().numpy()[0][feature_id]
        # feature = model(image)["concept_probs"].cpu().detach().numpy()[0][feature_id]

        for attr_id, attribute in enumerate(attributes):
            value_on = attribute * feature + (1 - attribute) * (1 - feature)
            attribute_values[attr_id][0] += value_on

            value_off = attribute * (1 - feature) + (1 - attribute) * feature
            attribute_values[attr_id][1] += value_off
    
    attribute_values_max = [max(p) for p in attribute_values]
    attribute_values_max = max(attribute_values_max)
    score = attribute_values_max / len(train_set)

    scores.append(score)

metric = np.array(scores).mean()

print(metric)

100%|██████████| 1845/1845 [00:42<00:00, 43.35it/s]

0.7840587777528067


Baseline: 0.7722590434662505

Framework: 0.7840587777528067

Baseline: 0.7481238945043126 / 0.8002787673469156 / 

Our framework: 0.733698356318566 / 0.733698356318566 / 0.77456 (w/o textual, anneal)


In [56]:
scores, np.array(scores).mean() # baseline

([0.8573726896832234,
  0.839452540636915,
  0.7797192060542814,
  0.8681423352541888,
  0.8260357244465,
  0.8895389913930329,
  0.8637059218046688,
  0.7592228388927755,
  0.797043976947101,
  0.7546556928753102],
 0.8234889917987998)

In [9]:
scores # framework

[0.7648627196583573,
 0.6768948672750362,
 0.7658918708248464,
 0.9917780249794046,
 0.8336742381045732,
 0.7635008184147333,
 0.7613273321342036,
 0.7518359585674752,
 0.6415101127414388,
 0.7674385555471499]

tensor([[ 0.0122, -0.0529, -0.0110,  ..., -0.0344, -0.0533,  0.0093],
        [ 0.0010,  0.0120,  0.0526,  ..., -0.0378,  0.0056, -0.0115],
        [-0.0123,  0.0306,  0.0034,  ..., -0.0043, -0.0447,  0.0298],
        ...,
        [ 0.0362,  0.0370, -0.0468,  ...,  0.0215,  0.0100,  0.0521],
        [ 0.0442,  0.0317, -0.0362,  ...,  0.0499, -0.0278,  0.0430],
        [ 0.0372, -0.0121,  0.0365,  ..., -0.0190, -0.0329, -0.0160]],

Predictor:  Parameter containing:
tensor([[-0.0137, -0.0356,  0.0175,  ..., -0.0315,  0.0064, -0.0237],
        [-0.0178,  0.0340,  0.0079,  ...,  0.0032, -0.0259, -0.0187],
        [ 0.0283, -0.0175, -0.0104,  ..., -0.0227, -0.0323, -0.0349],
        ...,
        [-0.0439,  0.0097, -0.0294,  ..., -0.0208,  0.0355,  0.0253],
        [-0.0421, -0.0303, -0.0202,  ..., -0.0343,  0.0074,  0.0075],
        [ 0.0195,  0.0237, -0.0137,  ..., -0.0427, -0.0188,  0.0273]],

In [16]:
import torch


isinstance(torch.nn.Linear(10, 5), torch.nn.Linear)

True